<a href="https://colab.research.google.com/github/thumb-chuu/BigData-Internship_statistics/blob/main/%EC%97%84%EC%A7%80%EC%88%98_%EC%8B%A4%EC%8A%B5(5)%EC%B9%B4%EC%9D%B4%EC%A0%9C%EA%B3%B1%EA%B2%80%EC%A0%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ◆ 통계적 검정 방법 실습


## 3. 카이제곱 검정


### 1) 적합도 검정(Goodness of Fit)
- k개의 범주(혹은 계급)을 가지는 한 개의 요인(factor)에 대해서 어떤 이론적 분포를 따르고 있는지를 검정하는 방법

stats.chisquare()
- 귀무가설 : 실제 분포와 이론적 분포 간에는 차이가 없다. (두 분포가 일치한다.)
- 대립가설 : 실제 분포와 이론적 분포 간에는 차이가 있다. (두 분포가 일치하지 않는다.)

In [1]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-nanum is already the newest version (20170925-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 62 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 10 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/var/cache/fontconfig: cleaning cache directory
/root/.cache/fontconfig: not cleaning non-existent

In [2]:
# 패키지 임포트
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)      # FutureWarning 제거

import pandas as pd
import numpy as np
import math
from scipy import stats

# 시각화 패키지
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# os에 따른 Font 깨짐 제거를 위해 Font 지정 
import os 
if os.name == 'nt' :
    font_family = "NanumBarunGothic" # Windows OS 
else :
    font_family = "NanumBarunGothic" # MAC OS

# - 값이 깨지는 문제 해결을 위해 파라미터 값 설정 
sns.set(font=font_family, rc ={"axes.unicode_minus" : False})

In [3]:
#### 적합도 검정 ####
## 데이터 생성
# 4교대 근무, 1일 불량 수 조사, 각조의 불량 수 차이 여부 검정
observed_, expected_ = [15, 35, 24, 26], [25, 25, 25, 25]
ct_1 = pd.DataFrame([observed_, expected_], columns=["A조", "B조", "C조", "D조"], index=['관측빈도', '기대빈도'])
ct_1

## 카이제곱 검정 통계량 계산
observed = ct_1.loc["관측빈도"]
expected = ct_1.loc["기대빈도"]

chi_squared_test = (observed-expected).pow(2)/expected

In [4]:
chi_squared_statistic = chi_squared_test.sum()
print('카이제곱 통계량', chi_squared_statistic)

## 카이제곱 검정 수행(적합도 검정)
dof = (4-1)*(2-1)
chi_ = stats.chi2(dof)
chi_05 = chi_.ppf(0.95)
print('기각역:', chi_05)

# 카이제곱 통계량 기반 검정
if chi_05 >= chi_squared_statistic:
    print("귀무가설 채택, 4개조의 불량수는 차이가 없다.")
else:
    print("귀무가설 기각, 4개조의 불량수는 차이가 있다.")

p_val = 1-chi_.cdf(chi_squared_statistic)

print('p-value:', p_val)

# 유의수준 기반 검정
if p_val >= 0.05:
    print("귀무가설 채택, 4개조의 불량수는 차이가 없다.")
else:
    print("귀무가설 기각, 4개조의 불량수는 차이가 있다.")

## chisquare: 모집단의 분포를 알고 있고, 이에 대해 검증할 때, 활용 package
stats.chisquare(ct_1.loc["관측빈도"].values, f_exp = ct_1.loc["기대빈도"].values)

카이제곱 통계량 8.079999999999998
기각역: 7.814727903251179
귀무가설 기각, 4개조의 불량수는 차이가 있다.
p-value: 0.04438695936738335
귀무가설 기각, 4개조의 불량수는 차이가 있다.


Power_divergenceResult(statistic=8.079999999999998, pvalue=0.044386959367383315)

### 2) 독립성 검정(Test of Independence)
- 모집단을 범주화하는 기준이 되는 두 변수 A, B가 서로 독립적으로 관측값에 영향을 미치는지의 여부를 검정하는 것

crosstab() : 교차표 생성 함수

scipy 모듈의 chi2_contingency() : 2개의 범주형 변수에 대한 카이제곱 통계량 계산

- 귀무가설 : 두 변수 사이에는 연관이 없다. (독립이다)
- 대립가설 : 두 변수 사이에는 연관이 있다. (종속이다)

In [5]:
# 교차분석하기 위한 범주형 데이터 정의
cross_data = pd.DataFrame(
                {"영양제": ['복용', '복용', '복용', '복용', '복용', '복용', '복용', '복용',
                            '복용', '복용', '복용', '복용', '복용', '복용', '복용', '복용',
                            '복용', '복용', '복용', '복용', 
                            '미복용', '미복용', '미복용', '미복용', '미복용', '미복용', '미복용', '미복용',
                            '미복용', '미복용', '미복용', '미복용', '미복용', '미복용', '미복용', '미복용',
                            '미복용', '미복용', '미복용', '미복용'],
                 "감기여부": ['유', '유', '유', '유', '유', '무', '무', '무', '무', '무',
                              '무', '무', '무', '무', '무', '무', '무', '무', '무', '무',
                              '유', '유', '유', '유', '유', '유', '유', '유', '유', '유',
                              '유', '유', '무', '무', '무', '무', '무', '무', '무', '무']
                 }
)

cross_data.head(5)

,영양제,감기여부
0,복용,유
1,복용,유
2,복용,유
3,복용,유
4,복용,유


In [6]:
# 교차분석표 생성
## summary 행렬 포함한 교차표
print(pd.crosstab(cross_data['영양제'], cross_data['감기여부'], margins=True))

## 계산하기 위한 교차표
data_crosstab = pd.crosstab(cross_data['영양제'], cross_data['감기여부'], margins=False)

감기여부   무   유  All
영양제              
미복용    8  12   20
복용    15   5   20
All   23  17   40


In [7]:
data_crosstab

감기여부,무,유
영양제,,
미복용,8,12
복용,15,5


In [14]:
# 교차표분석 : 카이제곱 통계량 산출
from scipy.stats import chi2_contingency

# correction=True를 적용하면 Yates' correction이 적용되어 검정 통계량이 보수적으로 더 낮게 나옴
result = chi2_contingency(observed=data_crosstab, correction=False)
print("1. 카이제곱 통계량:", result[0])
print("2. p-value:", result[1])
print("3. df:", result[2])                  # (행의 개수 - 1)*(열의 개수 - 1)
print("4. 기대값 행렬:") 
pd.DataFrame(result[3]).rename(index={0:'복용', 1:'미복용'}, columns={0:'유', 1:'무'})

if 0.05 >= result[1]:                       # p-value
    print("귀무가설 기각, 감기약 복용과 감기 유무는 차이가 있다.")
else:
    print("귀무가설 채택, 감기약 복용과 감기 유무는 차이가 없다.")

1. 카이제곱 통계량: 5.012787723785166
2. p-value: 0.025160759200408785
3. df: 1
4. 기대값 행렬:
귀무가설 기각, 감기약 복용과 감기 유무는 차이가 있다.
